In [ ]:
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd

import __module_import__
from dynamic_contagion import *
from market_conditions import *
from reinsurance import *

In [ ]:
# Other parameters
maturity = 3
markup = 0.4
R = int(1e6)
seed = 123

# Asset parameters
V_0 = 130
phi_V = -3  # * (1.3) # = V0 / L0
sigma_V = 0.05

# Liability parameters
L_0 = 100
phi_L = -3
sigma_L = 0.02

# Interest rate parameters
r_0 = 0.02
k = 0.2
m = 0.05  # 0.5
upsilon = 0.1
eta_r = -0.01

# Catastrophe loss size distribution parameters
mu_C = 2
sigma_C = 0.5

In [ ]:
# Poisson process
def simulate_poisson(seed):
    lambda_ = 0.5
    rg = rnd.default_rng(seed)
    return rg.poisson(lambda_ * maturity)


# Cox proces
def simulate_cox(seed):
    lambda0 = 0.49
    a = 0.4
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(seed, maturity, lambda0, a, rho, delta, 0.0, 0.0, 0.0, 0.5)


# Hawkes process
def simulate_hawkes(seed):
    lambda0 = 0.47
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.0)


# Dynamic contagion process
def simulate_dcp(seed):
    lambda0 = 0.29
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5)


simulators = (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp)

## Tables 1-4

In [ ]:
ROUNDING = 4

As = (10.0, 15.0, 20.0, 25.0, 30.0)
Ms = (60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0)


def prices_to_df(prices, As, Ms):
    try:
        cols = [f"$M={int(m)}$" for m in Ms]
        rows = [f"$A={int(a)}$" for a in As]
    except ValueError:
        cols = [f"$M={m}$" for m in Ms]
        rows = [f"$A={a}$" for a in As]

    return pd.DataFrame(prices.round(ROUNDING), columns=cols, index=rows)


def prices_to_tex(prices, As, Ms):
    df = prices_to_df(prices, As, Ms)
    display(df)
    return df.style.to_latex().replace("lrrrrrrr", "c|c|c|c|c|c|c|c").replace("00 ", " ")

In [ ]:
%%time
prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    A=As,
    M=Ms,
)

CPU times: user 26.8 s, sys: 932 ms, total: 27.7 s
Wall time: 27.7 s


In [ ]:
# Tables 1 to 4
cat_models = ("Poisson", "Cox", "Hawkes", "DCP")
for c in range(4):
    print(cat_models[c])
    print(prices_to_tex(prices[c], As, Ms))

Poisson


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.2633,7.2706,7.2742,7.2759,7.2767,7.2771,7.2773
$A=15$,4.5877,4.5956,4.5995,4.6013,4.6022,4.6026,4.6028
$A=20$,2.7921,2.8008,2.8050,2.8070,2.8079,2.8083,2.8086
$A=25$,1.6426,1.6519,1.6565,1.6587,1.6597,1.6602,1.6604
$A=30$,0.9380,0.9481,0.9531,0.9554,0.9565,0.9570,0.9573


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.2633 & 7.2706 & 7.2742 & 7.2759 & 7.2767 & 7.2771 & 7.2773 \\
$A=15$ & 4.5877 & 4.5956 & 4.5995 & 4.6013 & 4.6022 & 4.6026 & 4.6028 \\
$A=20$ & 2.7921 & 2.8008 & 2.8050 & 2.8070 & 2.8079 & 2.8083 & 2.8086 \\
$A=25$ & 1.6426 & 1.6519 & 1.6565 & 1.6587 & 1.6597 & 1.6602 & 1.6604 \\
$A=30$ & 0.9380 & 0.9481 & 0.9531 & 0.9554 & 0.9565 & 0.9570 & 0.9573 \\
\end{tabular}

Cox


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.2382,7.2462,7.2502,7.2521,7.2531,7.2536,7.2538
$A=15$,4.5959,4.6047,4.6090,4.6110,4.6121,4.6126,4.6129
$A=20$,2.8164,2.8260,2.8307,2.8329,2.8340,2.8346,2.8349
$A=25$,1.6727,1.6832,1.6883,1.6907,1.6920,1.6925,1.6928
$A=30$,0.9650,0.9763,0.9818,0.9845,0.9858,0.9864,0.9868


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.2382 & 7.2462 & 7.2502 & 7.2521 & 7.2531 & 7.2536 & 7.2538 \\
$A=15$ & 4.5959 & 4.6047 & 4.6090 & 4.6110 & 4.6121 & 4.6126 & 4.6129 \\
$A=20$ & 2.8164 & 2.8260 & 2.8307 & 2.8329 & 2.8340 & 2.8346 & 2.8349 \\
$A=25$ & 1.6727 & 1.6832 & 1.6883 & 1.6907 & 1.6920 & 1.6925 & 1.6928 \\
$A=30$ & 0.9650 & 0.9763 & 0.9818 & 0.9845 & 0.9858 & 0.9864 & 0.9868 \\
\end{tabular}

Hawkes


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.0884,8.1620,8.2134,8.2496,8.2754,8.2937,8.3069
$A=15$,5.9090,5.9889,6.0443,6.0831,6.1106,6.1301,6.1439
$A=20$,4.2961,4.3833,4.4435,4.4853,4.5147,4.5355,4.5502
$A=25$,3.0969,3.1919,3.2576,3.3031,3.3349,3.3571,3.3728
$A=30$,2.2032,2.3060,2.3775,2.4272,2.4617,2.4858,2.5026


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.0884 & 8.1620 & 8.2134 & 8.2496 & 8.2754 & 8.2937 & 8.3069 \\
$A=15$ & 5.9090 & 5.9889 & 6.0443 & 6.0831 & 6.1106 & 6.1301 & 6.1439 \\
$A=20$ & 4.2961 & 4.3833 & 4.4435 & 4.4853 & 4.5147 & 4.5355 & 4.5502 \\
$A=25$ & 3.0969 & 3.1919 & 3.2576 & 3.3031 & 3.3349 & 3.3571 & 3.3728 \\
$A=30$ & 2.2032 & 2.3060 & 2.3775 & 2.4272 & 2.4617 & 2.4858 & 2.5026 \\
\end{tabular}

DCP


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.1700,8.2422,8.2926,8.3277,8.3521,8.3696,8.3821
$A=15$,5.9639,6.0423,6.0967,6.1343,6.1603,6.1789,6.1921
$A=20$,4.3290,4.4146,4.4737,4.5142,4.5422,4.5620,4.5760
$A=25$,3.1150,3.2083,3.2728,3.3168,3.3470,3.3682,3.3831
$A=30$,2.2110,2.3118,2.3820,2.4301,2.4629,2.4857,2.5017


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.1700 & 8.2422 & 8.2926 & 8.3277 & 8.3521 & 8.3696 & 8.3821 \\
$A=15$ & 5.9639 & 6.0423 & 6.0967 & 6.1343 & 6.1603 & 6.1789 & 6.1921 \\
$A=20$ & 4.3290 & 4.4146 & 4.4737 & 4.5142 & 4.5422 & 4.5620 & 4.5760 \\
$A=25$ & 3.1150 & 3.2083 & 3.2728 & 3.3168 & 3.3470 & 3.3682 & 3.3831 \\
$A=30$ & 2.2110 & 2.3118 & 2.3820 & 2.4301 & 2.4629 & 2.4857 & 2.5017 \\
\end{tabular}



## Table 5: Default risk premium

In [ ]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
)[:, np.newaxis]

CPU times: user 25.5 s, sys: 738 ms, total: 26.2 s
Wall time: 26.2 s


In [ ]:
risky_prices.round(4)

array([[2.8086],
       [2.8349],
       [4.5502],
       [4.576 ]])

In [ ]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
)[:, np.newaxis]

CPU times: user 25.4 s, sys: 643 ms, total: 26 s
Wall time: 26 s


In [ ]:
safe_prices.round(4)

array([[2.8495],
       [2.879 ],
       [4.8205],
       [4.8414]])

In [ ]:
risk_premium = safe_prices - risky_prices
risk_premium.round(4)

array([[0.0409],
       [0.0441],
       [0.2703],
       [0.2654]])

In [ ]:
default_risk_premiums = np.hstack((risky_prices, safe_prices, risk_premium)).round(4)
default_risk_premiums

array([[2.8086, 2.8495, 0.0409],
       [2.8349, 2.879 , 0.0441],
       [4.5502, 4.8205, 0.2703],
       [4.576 , 4.8414, 0.2654]])

In [ ]:
print(pd.DataFrame(default_risk_premiums).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrr}
 & 0 & 1 & 2 \\
0 & 2.8086 & 2.8495 & 0.0409 \\
1 & 2.8349 & 2.8790 & 0.0441 \\
2 & 4.5502 & 4.8205 & 0.2703 \\
3 & 4.5760 & 4.8414 & 0.2654 \\
\end{tabular}



## Table 6: Impacts of externally-excited jump frequency rate

In [ ]:
def simulate_dcp_variations(seed, rho):
    lambda0 = 0.29
    a = 0.26
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5)


dcp_variations = [partial(simulate_dcp_variations, rho=rho) for rho in (0.4, 3, 10, 20)]

In [ ]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
).T

CPU times: user 1min 22s, sys: 2.93 s, total: 1min 25s
Wall time: 1min 25s


In [ ]:
risky_prices.round(4)

array([[ 4.137 ,  4.576 ,  4.7667],
       [13.5344, 15.2673, 16.2062],
       [39.8862, 46.4095, 51.3561],
       [56.6998, 66.8516, 76.0095]])

In [ ]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

CPU times: user 27.1 s, sys: 805 ms, total: 27.9 s
Wall time: 27.9 s


In [ ]:
safe_prices.round(4)

array([[ 4.8414,  4.8414,  4.8414],
       [16.6552, 16.6552, 16.6552],
       [54.5809, 54.5809, 54.5809],
       [83.1984, 83.1984, 83.1984]])

In [ ]:
risk_premium = safe_prices - risky_prices

In [ ]:
impacts_of_ext_jump_freq = np.hstack([risky_prices, risk_premium]).round(4)
impacts_of_ext_jump_freq

array([[4.13700e+00, 4.57600e+00, 4.76670e+00, 7.04400e-01, 2.65400e-01,
        7.47000e-02],
       [1.35344e+01, 1.52673e+01, 1.62062e+01, 3.12080e+00, 1.38790e+00,
        4.49000e-01],
       [3.98862e+01, 4.64095e+01, 5.13561e+01, 1.46947e+01, 8.17130e+00,
        3.22480e+00],
       [5.66998e+01, 6.68516e+01, 7.60095e+01, 2.64986e+01, 1.63468e+01,
        7.18890e+00]])

In [ ]:
print(pd.DataFrame(impacts_of_ext_jump_freq).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrrrr}
 & 0 & 1 & 2 & 3 & 4 & 5 \\
0 & 4.1370 & 4.5760 & 4.7667 & 0.7044 & 0.2654 & 0.0747 \\
1 & 13.5344 & 15.2673 & 16.2062 & 3.1208 & 1.3879 & 0.4490 \\
2 & 39.8862 & 46.4095 & 51.3561 & 14.6947 & 8.1713 & 3.2248 \\
3 & 56.6998 & 66.8516 & 76.0095 & 26.4986 & 16.3468 & 7.1889 \\
\end{tabular}



## Table 7: Impacts of externally-excited jump magnitude

In [ ]:
def simulate_dcp_variations(seed, mu_F=0.25, mu_G=0.5):
    lambda0 = 0.29
    a = 0.26
    delta = 1
    rho = 0.4

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 2 * mu_G, 0.0, 2 * mu_F
    )


dcp_variations = [partial(simulate_dcp_variations, mu_F=mu_F) for mu_F in (0.25, 1.0, 4.0, 8.0)]

In [ ]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

CPU times: user 1min 21s, sys: 3.27 s, total: 1min 24s
Wall time: 1min 24s


In [ ]:
impacts_of_ext_jump_mag = np.hstack([risky_prices, risk_premium]).round(4)
impacts_of_ext_jump_mag

array([[ 4.137 ,  4.576 ,  4.7667,  0.7044,  0.2654,  0.0747],
       [ 8.5331,  9.5955, 10.1627,  1.9013,  0.8389,  0.2717],
       [23.304 , 27.0421, 29.9316,  8.6188,  4.8807,  1.9912],
       [31.4876, 36.8698, 41.4919, 13.5394,  8.1572,  3.5351]])

In [ ]:
print(pd.DataFrame(impacts_of_ext_jump_mag).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrrrr}
 & 0 & 1 & 2 & 3 & 4 & 5 \\
0 & 4.1370 & 4.5760 & 4.7667 & 0.7044 & 0.2654 & 0.0747 \\
1 & 8.5331 & 9.5955 & 10.1627 & 1.9013 & 0.8389 & 0.2717 \\
2 & 23.3040 & 27.0421 & 29.9316 & 8.6188 & 4.8807 & 1.9912 \\
3 & 31.4876 & 36.8698 & 41.4919 & 13.5394 & 8.1572 & 3.5351 \\
\end{tabular}



## Table 8: Impacts of self-excited jump magnitude

In [ ]:
%%time
dcp_variations = [partial(simulate_dcp_variations, mu_G=mu_G) for mu_G in (0.5, 1.0, 2.0, 3.0)]

risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    dcp_variations,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

CPU times: user 1min 43s, sys: 2.91 s, total: 1min 46s
Wall time: 1min 46s


In [ ]:
impact_of_self_jump_mag = np.hstack([risky_prices, risk_premium]).round(4)
impact_of_self_jump_mag

array([[ 4.137 ,  4.576 ,  4.7667,  0.7044,  0.2654,  0.0747],
       [ 9.145 , 10.4484, 11.3248,  2.7225,  1.4191,  0.5427],
       [19.0516, 22.2658, 25.0118,  8.0721,  4.858 ,  2.112 ],
       [24.8796, 29.2425, 33.1769, 11.4628,  7.0999,  3.1656]])

In [ ]:
print(pd.DataFrame(impact_of_self_jump_mag).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrrrr}
 & 0 & 1 & 2 & 3 & 4 & 5 \\
0 & 4.1370 & 4.5760 & 4.7667 & 0.7044 & 0.2654 & 0.0747 \\
1 & 9.1450 & 10.4484 & 11.3248 & 2.7225 & 1.4191 & 0.5427 \\
2 & 19.0516 & 22.2658 & 25.0118 & 8.0721 & 4.8580 & 2.1120 \\
3 & 24.8796 & 29.2425 & 33.1769 & 11.4628 & 7.0999 & 3.1656 \\
\end{tabular}



## Tables 9-12: Prices with catbond

In [ ]:
K = 40.0
F = 25.0

In [ ]:
%%time
catbond_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    A=As,
    M=Ms,
    catbond=True,
    K=K,
    F=F,
)

CPU times: user 26.4 s, sys: 682 ms, total: 27 s
Wall time: 27.1 s


In [ ]:
print("Prices of reinsurance when the reinsurer issues a catbond\n")

for c in range(4):
    print(cat_models[c])
    print(prices_to_tex(catbond_prices[c], As, Ms))

Prices of reinsurance when the reinsurer issues a catbond

Poisson


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.3128,7.3231,7.3282,7.3305,7.3316,7.3321,7.3324
$A=15$,4.6226,4.6335,4.6390,4.6416,4.6428,4.6433,4.6436
$A=20$,2.8142,2.8256,2.8314,2.8343,2.8356,2.8362,2.8365
$A=25$,1.6549,1.6665,1.6726,1.6756,1.6770,1.6777,1.6780
$A=30$,0.9439,0.9557,0.9619,0.9650,0.9666,0.9673,0.9677


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.3128 & 7.3231 & 7.3282 & 7.3305 & 7.3316 & 7.3321 & 7.3324 \\
$A=15$ & 4.6226 & 4.6335 & 4.6390 & 4.6416 & 4.6428 & 4.6433 & 4.6436 \\
$A=20$ & 2.8142 & 2.8256 & 2.8314 & 2.8343 & 2.8356 & 2.8362 & 2.8365 \\
$A=25$ & 1.6549 & 1.6665 & 1.6726 & 1.6756 & 1.6770 & 1.6777 & 1.6780 \\
$A=30$ & 0.9439 & 0.9557 & 0.9619 & 0.9650 & 0.9666 & 0.9673 & 0.9677 \\
\end{tabular}

Cox


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,7.2916,7.3029,7.3085,7.3112,7.3125,7.3131,7.3135
$A=15$,4.6336,4.6457,4.6518,4.6547,4.6562,4.6568,4.6572
$A=20$,2.8405,2.8532,2.8596,2.8628,2.8644,2.8652,2.8655
$A=25$,1.6864,1.6995,1.7062,1.7096,1.7113,1.7121,1.7125
$A=30$,0.9718,0.9850,0.9919,0.9954,0.9973,0.9982,0.9986


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 7.2916 & 7.3029 & 7.3085 & 7.3112 & 7.3125 & 7.3131 & 7.3135 \\
$A=15$ & 4.6336 & 4.6457 & 4.6518 & 4.6547 & 4.6562 & 4.6568 & 4.6572 \\
$A=20$ & 2.8405 & 2.8532 & 2.8596 & 2.8628 & 2.8644 & 2.8652 & 2.8655 \\
$A=25$ & 1.6864 & 1.6995 & 1.7062 & 1.7096 & 1.7113 & 1.7121 & 1.7125 \\
$A=30$ & 0.9718 & 0.9850 & 0.9919 & 0.9954 & 0.9973 & 0.9982 & 0.9986 \\
\end{tabular}

Hawkes


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.3079,8.4123,8.4851,8.5351,8.5693,8.5929,8.6092
$A=15$,6.0684,6.1794,6.2582,6.3133,6.3512,6.3772,6.3950
$A=20$,4.4015,4.5171,4.6009,4.6605,4.7023,4.7310,4.7506
$A=25$,3.1593,3.2776,3.3648,3.4281,3.4733,3.5049,3.5267
$A=30$,2.2359,2.3553,2.4444,2.5104,2.5584,2.5926,2.6165


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.3079 & 8.4123 & 8.4851 & 8.5351 & 8.5693 & 8.5929 & 8.6092 \\
$A=15$ & 6.0684 & 6.1794 & 6.2582 & 6.3133 & 6.3512 & 6.3772 & 6.3950 \\
$A=20$ & 4.4015 & 4.5171 & 4.6009 & 4.6605 & 4.7023 & 4.7310 & 4.7506 \\
$A=25$ & 3.1593 & 3.2776 & 3.3648 & 3.4281 & 3.4733 & 3.5049 & 3.5267 \\
$A=30$ & 2.2359 & 2.3553 & 2.4444 & 2.5104 & 2.5584 & 2.5926 & 2.6165 \\
\end{tabular}

DCP


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,8.3890,8.4915,8.5629,8.6113,8.6438,8.6663,8.6818
$A=15$,6.1227,6.2318,6.3093,6.3625,6.3985,6.4232,6.4401
$A=20$,4.4338,4.5474,4.6298,4.6875,4.7271,4.7544,4.7730
$A=25$,3.1769,3.2931,3.3788,3.4401,3.4831,3.5131,3.5337
$A=30$,2.2431,2.3605,2.4480,2.5117,2.5574,2.5900,2.6127


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 8.3890 & 8.4915 & 8.5629 & 8.6113 & 8.6438 & 8.6663 & 8.6818 \\
$A=15$ & 6.1227 & 6.2318 & 6.3093 & 6.3625 & 6.3985 & 6.4232 & 6.4401 \\
$A=20$ & 4.4338 & 4.5474 & 4.6298 & 4.6875 & 4.7271 & 4.7544 & 4.7730 \\
$A=25$ & 3.1769 & 3.2931 & 3.3788 & 3.4401 & 3.4831 & 3.5131 & 3.5337 \\
$A=30$ & 2.2431 & 2.3605 & 2.4480 & 2.5117 & 2.5574 & 2.5900 & 2.6127 \\
\end{tabular}



## Tables 13-16: Percentage increase in costs after issuing a catbond

In [ ]:
print("Percentage increase in price for reinsurance after the reinsurer issues a catbond\n")

for c in range(4):
    print(cat_models[c])
    percentage_increase = ((catbond_prices[c] - prices[c]) / prices[c] * 100).round(2)
    print(prices_to_tex(percentage_increase, As, Ms).replace("00 ", "\% "))

Percentage increase in price for reinsurance after the reinsurer issues a catbond

Poisson


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,0.68,0.72,0.74,0.75,0.75,0.76,0.76
$A=15$,0.76,0.82,0.86,0.87,0.88,0.88,0.88
$A=20$,0.79,0.89,0.94,0.97,0.99,0.99,0.99
$A=25$,0.75,0.88,0.97,1.02,1.05,1.06,1.06
$A=30$,0.64,0.80,0.92,1.00,1.05,1.07,1.08


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 0.68\% & 0.72\% & 0.74\% & 0.75\% & 0.75\% & 0.76\% & 0.76\% \\
$A=15$ & 0.76\% & 0.82\% & 0.86\% & 0.87\% & 0.88\% & 0.88\% & 0.88\% \\
$A=20$ & 0.79\% & 0.89\% & 0.94\% & 0.97\% & 0.99\% & 0.99\% & 0.99\% \\
$A=25$ & 0.75\% & 0.88\% & 0.97\% & 1.02\% & 1.05\% & 1.06\% & 1.06\% \\
$A=30$ & 0.64\% & 0.80\% & 0.92\% & 1.00\% & 1.05\% & 1.07\% & 1.08\% \\
\end{tabular}

Cox


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,0.74,0.78,0.80,0.81,0.82,0.82,0.82
$A=15$,0.82,0.89,0.93,0.95,0.96,0.96,0.96
$A=20$,0.85,0.96,1.02,1.06,1.07,1.08,1.08
$A=25$,0.82,0.97,1.06,1.12,1.14,1.16,1.16
$A=30$,0.71,0.89,1.03,1.11,1.16,1.19,1.20


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 0.74\% & 0.78\% & 0.80\% & 0.81\% & 0.82\% & 0.82\% & 0.82\% \\
$A=15$ & 0.82\% & 0.89\% & 0.93\% & 0.95\% & 0.96\% & 0.96\% & 0.96\% \\
$A=20$ & 0.85\% & 0.96\% & 1.02\% & 1.06\% & 1.07\% & 1.08\% & 1.08\% \\
$A=25$ & 0.82\% & 0.97\% & 1.06\% & 1.12\% & 1.14\% & 1.16\% & 1.16\% \\
$A=30$ & 0.71\% & 0.89\% & 1.03\% & 1.11\% & 1.16\% & 1.19\% & 1.20\% \\
\end{tabular}

Hawkes


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,2.71,3.07,3.31,3.46,3.55,3.61,3.64
$A=15$,2.70,3.18,3.54,3.78,3.94,4.03,4.09
$A=20$,2.45,3.05,3.54,3.91,4.15,4.31,4.40
$A=25$,2.02,2.68,3.29,3.78,4.15,4.40,4.56
$A=30$,1.48,2.14,2.81,3.43,3.93,4.30,4.55


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 2.71\% & 3.07\% & 3.31\% & 3.46\% & 3.55\% & 3.61\% & 3.64\% \\
$A=15$ & 2.70\% & 3.18\% & 3.54\% & 3.78\% & 3.94\% & 4.03\% & 4.09\% \\
$A=20$ & 2.45\% & 3.05\% & 3.54\% & 3.91\% & 4.15\% & 4.31\% & 4.40\% \\
$A=25$ & 2.02\% & 2.68\% & 3.29\% & 3.78\% & 4.15\% & 4.40\% & 4.56\% \\
$A=30$ & 1.48\% & 2.14\% & 2.81\% & 3.43\% & 3.93\% & 4.30\% & 4.55\% \\
\end{tabular}

DCP


,$M=60$,$M=65$,$M=70$,$M=75$,$M=80$,$M=85$,$M=90$
$A=10$,2.68,3.03,3.26,3.41,3.49,3.54,3.58
$A=15$,2.66,3.14,3.49,3.72,3.87,3.95,4.00
$A=20$,2.42,3.01,3.49,3.84,4.07,4.22,4.31
$A=25$,1.99,2.64,3.24,3.72,4.07,4.30,4.45
$A=30$,1.45,2.10,2.77,3.36,3.84,4.19,4.43


\begin{tabular}{c|c|c|c|c|c|c|c}
 & $M=60$ & $M=65$ & $M=70$ & $M=75$ & $M=80$ & $M=85$ & $M=90$ \\
$A=10$ & 2.68\% & 3.03\% & 3.26\% & 3.41\% & 3.49\% & 3.54\% & 3.58\% \\
$A=15$ & 2.66\% & 3.14\% & 3.49\% & 3.72\% & 3.87\% & 3.95\% & 4.00\% \\
$A=20$ & 2.42\% & 3.01\% & 3.49\% & 3.84\% & 4.07\% & 4.22\% & 4.31\% \\
$A=25$ & 1.99\% & 2.64\% & 3.24\% & 3.72\% & 4.07\% & 4.30\% & 4.45\% \\
$A=30$ & 1.45\% & 2.10\% & 2.77\% & 3.36\% & 3.84\% & 4.19\% & 4.43\% \\
\end{tabular}



## Table 17: Default risk premium with catbond

In [ ]:
%%time

K = 40.0
F = 25.0

risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
)[:, np.newaxis]

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
)[:, np.newaxis]

catbond_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    catbond=True,
    K=K,
    F=F,
)[:, np.newaxis]



CPU times: user 1min 15s, sys: 1.72 s, total: 1min 17s
Wall time: 1min 17s


In [ ]:
catbond_premium = catbond_prices - risky_prices
catbond_premium.round(4)

array([[0.0279],
       [0.0306],
       [0.2004],
       [0.197 ]])

In [ ]:
catbond_risk_premiums = np.hstack((risky_prices, catbond_prices, safe_prices, catbond_premium)).round(4)
catbond_risk_premiums

array([[2.8086, 2.8365, 2.8495, 0.0279],
       [2.8349, 2.8655, 2.879 , 0.0306],
       [4.5502, 4.7506, 4.8205, 0.2004],
       [4.576 , 4.773 , 4.8414, 0.197 ]])

In [ ]:
print(pd.DataFrame(catbond_risk_premiums).style.to_latex().replace("00 ", " "))

\begin{tabular}{lrrrr}
 & 0 & 1 & 2 & 3 \\
0 & 2.8086 & 2.8365 & 2.8495 & 0.0279 \\
1 & 2.8349 & 2.8655 & 2.8790 & 0.0306 \\
2 & 4.5502 & 4.7506 & 4.8205 & 0.2004 \\
3 & 4.5760 & 4.7730 & 4.8414 & 0.1970 \\
\end{tabular}

